In [ ]:
#######################################################################################
# Author: Srijan Verma                                                              #
# School of Pharmacy                                                                #
# Sirimulla Research Group [http://www.sirimullaresearchgroup.com/]                 #
# The University of Texas at El Paso, TX, USA                                       #
# Last modified: 19/12/2019                                                         #
# Copyright (c) 2019 Srijan Verma and Sirimulla Research Group, under MIT license   #
#######################################################################################

In [29]:
import pandas as pd
from tqdm import tqdm_notebook
from glob import glob
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm_notebook
import pandas as pd
from glob import glob
import time
import os

In [5]:
enzyme = ['3a4', '2d6', '2c9']
for enz in tqdm_notebook(enzyme):
    interesting_files = glob("../dataset/sub_inhib_2005_paper_data/data_with_smiles/"+enz+"/*.csv")  
    df = pd.concat((pd.read_csv(f, header = 0, index_col = 0) for f in (interesting_files))) 
#     print(len(df))
    df_deduplicated = df.drop_duplicates() 
    df_deduplicated.reset_index(drop=True,inplace=True)
    df_deduplicated.drop(['Class', 'Set'], axis=1, inplace=True)
    df_deduplicated.columns = ['drug_name', 'Smiles', 'drug_type']
    df_deduplicated = df_deduplicated[['drug_name', 'drug_type', 'Smiles']]
    df_deduplicated.to_csv('../dataset/sub_inhib_2005_paper_data/data_with_smiles/'+enz+'.csv')


/Users/pankajverma/anaconda3/envs/seqcomhol/lib/python3.6/site-packages/pandas/core/frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [30]:
df_tran = pd.read_csv('../dataset/transformer_plus_paper_data/data_with_smiles/2D6.csv', index_col=0)
df_paper = pd.read_csv('../dataset/sub_inhib_2005_paper_data/data_with_smiles/2d6.csv', index_col=0)
df = pd.concat([df_tran , df_paper], axis=0)
df = df.drop_duplicates() 
df.reset_index(drop=True,inplace=True)
df.to_csv('../dataset/two_paper_plus_transformer_combined/combined_data_with_smiles/2D6.csv')

In [31]:
df_tran = pd.read_csv('../dataset/transformer_plus_paper_data/data_with_smiles/2C9.csv', index_col=0)
df_paper = pd.read_csv('../dataset/sub_inhib_2005_paper_data/data_with_smiles/2c9.csv', index_col=0)
df = pd.concat([df_tran , df_paper], axis=0)
df = df.drop_duplicates() 
df.reset_index(drop=True,inplace=True)
df.to_csv('../dataset/two_paper_plus_transformer_combined/combined_data_with_smiles/2C9.csv')

In [32]:
df_tran = pd.read_csv('../dataset/transformer_plus_paper_data/data_with_smiles/3A4.csv', index_col=0)
df_paper = pd.read_csv('../dataset/sub_inhib_2005_paper_data/data_with_smiles/3a4.csv', index_col=0)
df = pd.concat([df_tran , df_paper], axis=0)
df = df.drop_duplicates() 
df.reset_index(drop=True,inplace=True)
df.to_csv('../dataset/two_paper_plus_transformer_combined/combined_data_with_smiles/3A4.csv')

In [6]:
# df['drug_type'] = df['drug_type'] +  '-'
# df

In [18]:
for csv in tqdm_notebook(glob('../dataset/two_paper_plus_transformer_combined/combined_data_with_smiles/*csv')):
    out = os.path.splitext(os.path.basename(csv))[0]
    df = pd.read_csv(csv, index_col = 0)
    
    for i in range(len(df)):
        if df['drug_type'][i] == '-':
            continue
        else:
            df['drug_type'][i] = out + '-' + df['drug_type'][i]
            
    df.to_csv('../dataset/two_paper_plus_transformer_combined/correct_label_with_smiles/'+out+'.csv')

In [19]:
interesting_files = glob("../dataset/two_paper_plus_transformer_combined/correct_label_with_smiles/*.csv")  
df = pd.concat((pd.read_csv(f, header = 0, index_col=0) for f in tqdm_notebook(interesting_files))) 
df_deduplicated = df.drop_duplicates() 
df_deduplicated.reset_index(drop=True, inplace=True)
df_deduplicated.to_csv('../dataset/two_paper_plus_transformer_combined/cleaned_v1.csv')

In [36]:
df_deduplicated = pd.read_csv('../dataset/two_paper_plus_transformer_combined/cleaned_v1.csv',index_col=0)

In [37]:
target_list = list(df_deduplicated['drug_type'].unique())

In [38]:
df_deduplicated.head(2)

,drug_name,drug_type,Smiles
0,Ciprofibrate,4A11-Inducer,CC(C)(OC1=CC=C(C=C1)[C@H]2CC2(Cl)Cl)C(O)=O
1,Clofibrate,4A11-Substrate,CCOC(=O)C(C)(C)OC1=CC=C(Cl)C=C1


In [39]:
smi = df_deduplicated['Smiles'].tolist()
len(smi)

8041

In [40]:
%%time
not_found = 0
can_smi = []
from rdkit import Chem
for smi in tqdm_notebook(smi):
    
    try:
        mol = Chem.MolFromSmiles(smi)
        cans = Chem.MolToSmiles(mol, True)
        can_smi.append(cans)
        
    except:
        can_smi.append('-')
        not_found = not_found + 1
        pass

# assert cans[0] == cans[1]


CPU times: user 4.94 s, sys: 80.8 ms, total: 5.02 s
Wall time: 5.7 s


In [41]:
not_found

8

In [42]:
len(can_smi)

8041

In [43]:
df_deduplicated['Canonical_Smiles'] = can_smi

In [45]:
df_deduplicated.drop(df_deduplicated.loc[df_deduplicated['Canonical_Smiles']=='-'].index, inplace=True)
df_deduplicated.reset_index(drop=True,inplace=True)

In [46]:
dr_name = []
dr_type = []
smi = []
# label = []

In [47]:
df = df_deduplicated
for k in tqdm_notebook(target_list):
    
    if k == '-':
        continue
        
    else:
    
        label = []

        for i in (range(len(df))):

            if df['drug_type'][i] == k:
                label.append('positive')
                continue

            else:
                label.append('negative')
                continue

        df['Label'] = label

        df_all = df

        df_all.reset_index(drop = True, inplace = True)

        drop_index = []

        for i in (range(len(df_all))):

            if df_all['drug_type'][i] == k:
                continue

            else:
                drop_index.append(i)
                continue

        df_p = df_all.drop(drop_index, axis = 0, inplace = False)
        df_p.reset_index(drop = True, inplace = True)

        df_pos = df_p.drop_duplicates(['Canonical_Smiles'])

        uni_pos_smiles = set(df_pos['Canonical_Smiles'].tolist())

        drop_index = []
        for i in (range(len(df_all))):

            if df_all['Canonical_Smiles'][i] in uni_pos_smiles:
                drop_index.append(i)
                continue

            else:
                continue

        df_neg = df_all.drop(drop_index, axis = 0, inplace = False)
        df_neg.reset_index(drop = True, inplace = True)

        df_neg = df_neg.drop_duplicates(['Canonical_Smiles'])

    #########################<GETTING FAM-SMILES-COUNT>####################################
        df_neg.reset_index(drop = True, inplace = True)
        df_pos.reset_index(drop = True, inplace = True)

    #     df_neg_index = df_neg.set_index('Final_Family')
    #     df_pos_index = df_pos.set_index('Final_Family')

    #     for fam in tqdm_notebook(fin_fam):
    #         get_fam_count(k, fam, df_neg_index, df_pos_index) 
    #######################################################################################

        clean_data = pd.concat([df_neg, df_pos], ignore_index=True, sort =True)

        if k.split('-')[1] == 'Inhibitor':
            clean_data.to_csv('../dataset/two_paper_plus_transformer_combined/inhib_data/' + k.split('-')[0] + '_inh.csv')
            continue

        elif k.split('-')[1] == 'Substrate':
            clean_data.to_csv('../dataset/two_paper_plus_transformer_combined/sub_data/' + k.split('-')[0] + '_sub.csv')
            
        else:
            clean_data.to_csv('../dataset/two_paper_plus_transformer_combined/inducer_data/' + k.split('-')[0] + '_ind.csv')
            


In [48]:
stats = ['inhib_data', 'sub_data', 'inducer_data']
for st in tqdm_notebook(stats):
    target = []
    dataset_size = []
    actives = []
    inactives = []
    
    for csv in glob('../dataset/two_paper_plus_transformer_combined/'+st+'/*csv'):
        target.append(os.path.splitext(os.path.basename(csv))[0])
        df = pd.read_csv(csv,index_col = 0)
        dataset_size.append(len(df))
        actives.append(len(df[df.Label == 'positive']))
        inactives.append(len(df[df.Label == 'negative']))
    
    df = pd.DataFrame(target, columns=['Target'])
    df['Dataset_Size'] = dataset_size
    df['Actives'] = actives
    df['Inactives'] = inactives
    
    df.sort_values('Target', ascending = True, inplace = True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv('../dataset/two_paper_plus_transformer_combined/stats/'+st+'_stats.csv')
        

In [49]:
interesting_files = glob("../dataset/two_paper_plus_transformer_combined/stats/*.csv")  
df = pd.concat((pd.read_csv(f, header = 0, index_col=0) for f in tqdm_notebook(interesting_files))) 
df_deduplicated = df.drop_duplicates() 
df_deduplicated.reset_index(drop=True, inplace=True)
df_deduplicated.sort_values('Target', ascending = True, inplace = True)
df_deduplicated.reset_index(drop=True, inplace=True)
df_deduplicated.to_csv('../dataset/two_paper_plus_transformer_combined/stats/inh-sub-ind-combined_stats.csv')

In [23]:
# a.drop(a.loc[a['Canonical_Smiles']=='-'].index, inplace=True)

In [50]:
a = pd.read_csv('/Users/pankajverma/Desktop/SOP/DixitSirProject/thesis/suman_sir_protein_subcell_project/dataset/pdsp_pharos_combined/individual_files_can_smi/actin_filaments.csv')

In [51]:
len(a)

225049